<a href="https://colab.research.google.com/github/Rohan641/Foreign_ExchangeRate_Forecast/blob/master/Forex_Forecasting.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [0]:
#Install the libraries
!pip install numpy
!pip install pandas
!pip install matplotlib

In [0]:
#Load all the reqd libs


import numpy as np

import pandas as pd

import matplotlib.pyplot as plt


    

In [5]:
from google.colab import files

uploaded = files.upload()

Saving data.xlsx to data.xlsx


In [16]:
# Reads an Excel file 

# return: data = ['date', 'last price']


def loadExcelData(fileName, sheetName):

  data = pd.DataFrame(pd.read_excel(fileName, sheetName), columns= ['Date', 'Last Price']) # creates a dataframe on reading excel files
	
  return data.values


cny_data = loadExcelData('data.xlsx','USDCNY')

jpy_data = loadExcelData('data.xlsx','USDJPY')

eur_data = loadExcelData('data.xlsx','USDEUR')

gbp_data = loadExcelData('data.xlsx','USDGBP')



array([[Timestamp('2005-07-21 00:00:00'), 8.11],
       [Timestamp('2005-07-22 00:00:00'), 8.1111],
       [Timestamp('2005-07-25 00:00:00'), 8.1097],
       ...,
       [Timestamp('2019-05-07 00:00:00'), 6.7763],
       [Timestamp('2019-05-08 00:00:00'), 6.7828],
       [Timestamp('2019-05-09 00:00:00'), 6.8274]], dtype=object)

In [0]:
#Return values(r2, r3, r4, ..., rn)


# construct rs
# r0 = None
# start from r1 = 100*log(p1-p0)


def construct_return(data):

  
  p = data[:,1].astype(float) # this line converts exchange price to float type
	
  r = np.zeros_like(p) # this creates an empty np obj of size p i.e 3521
	
  r[0] = None # set first elm to None
	
  r[1:] = 100.0 * (np.log(p[1:])-np.log(p[:-1])) # this helps in computing return
	
  return r

cny_r = construct_return(cny_data)
jpy_r = construct_return(jpy_data)
eur_r = construct_return(eur_data)
gbp_r = construct_return(gbp_data)



#p = cny_data[:,1].astype(float) # this line converts exchange price to float type

#r = np.zeros_like(p) # this creates an empty np obj of size p i.e 3521


In [30]:
# for example: SMA5

# in the form of (r2, (r2+r3)/2, (r2+r3+r4)/3, (r2+r3+r4+r5)/4, (r2+r3+r4+r5+r6)/5, 

# (r3+r4+r5+r6+r7)/5, ...)

def construct_SMA(r,degree):

  
  num_days = r.shape[0] # this actually gives the length of r
	
  sma = np.zeros_like(r) # now you create another empty obj of r shape
	
  for i in range(num_days):
	
    if i == 0:  # this is for the first element
	
        sma[i] = None
	
    elif i < degree:
	
        sma[i] = np.mean(r[1:i+1])# this is to calc moving avg 
	
    else:
	
        sma[i] = np.mean(r[i-degree+1:i+1]) # this after first iteration like if i > degree, use (i-degree+1)
	
  
  return sma



	# construct sma
cny_sma5 = construct_SMA(cny_r,5)
jpy_sma5 = construct_SMA(jpy_r,5)
eur_sma5 = construct_SMA(eur_r,5)
gbp_sma5 = construct_SMA(gbp_r,5)

  
cny_sma5

array([        nan,  0.01356258, -0.0018496 , ...,  0.12286233,
        0.14352253,  0.27400719])

In [0]:
# return last day's closing price
def construct_lastDay(r):

  lDay = np.zeros_like(r)
	
  lDay[0:2] = None
	
  lDay[2:] = r[1:-1]
	
  return lDay



	# construct last-day-closing-price structure
cny_lDay = construct_lastDay(cny_r)

jpy_lDay = construct_lastDay(jpy_r)

eur_lDay = construct_lastDay(eur_r)

gbp_lDay = construct_lastDay(gbp_r)



In [0]:


# The main method
def main():
	# load data = ['date', 'last price']
	cny_data = loadExcelData('data.xlsx','USDCNY')
	jpy_data = loadExcelData('data.xlsx','USDJPY')
	eur_data = loadExcelData('data.xlsx','USDEUR')
	gbp_data = loadExcelData('data.xlsx','USDGBP')
	
	# Fill data in the empty dates (weekends) with the average
	# of porior and posterior data
	# cny_data = fillEmptyDates(cny_data)
	# jpy_data = fillEmptyDates(jpy_data)
	# eur_data = fillEmptyDates(eur_data)
	# gbp_data = fillEmptyDates(gbp_data)

	# construct retrun values (r2,...,rn)
	cny_r = construct_return(cny_data)
	jpy_r = construct_return(jpy_data)
	eur_r = construct_return(eur_data)
	gbp_r = construct_return(gbp_data)

	# construct sma
	cny_sma5 = construct_SMA(cny_r,5)
	jpy_sma5 = construct_SMA(jpy_r,5)
	eur_sma5 = construct_SMA(eur_r,5)
	gbp_sma5 = construct_SMA(gbp_r,5)

	# construct last-day-closing-price structure
	cny_lDay = construct_lastDay(cny_r)
	jpy_lDay = construct_lastDay(jpy_r)
	eur_lDay = construct_lastDay(eur_r)
	gbp_lDay = construct_lastDay(gbp_r)

	# construct y 
	cny_y = construct_y(cny_data)
	jpy_y = construct_y(jpy_data)	
	eur_y = construct_y(eur_data)
	gbp_y = construct_y(gbp_data)

	# plot p
	plt.figure(2)
	plt.subplot(221)
	plt.plot(cny_data[:,1])
	plt.title('CNY')
	plt.subplot(222)
	plt.plot(jpy_data[:,1])
	plt.title('JPY')
	plt.subplot(223)
	plt.plot(eur_data[:,1])
	plt.title('EUR')
	plt.subplot(224)
	plt.plot(gbp_data[:,1])
	plt.title('GBP')
	plt.savefig('P.png')

	# plot r
	plt.figure(3)
	plt.subplot(221)
	plt.plot(cny_r)
	plt.title('CNY')
	plt.subplot(222)
	plt.plot(jpy_r)
	plt.title('JPY')
	plt.subplot(223)
	plt.plot(eur_r)
	plt.title('EUR')
	plt.subplot(224)

	plt.plot(gbp_r)
	plt.title('GBP')
	plt.savefig('R.png')

	# plot SMA5
	plt.figure(4)
	plt.subplot(221)
	plt.plot(cny_sma5)
	plt.title('CNY')
	plt.subplot(222)
	plt.plot(jpy_sma5)
	plt.title('JPY')
	plt.subplot(223)
	plt.plot(eur_sma5)
	plt.title('EUR')
	plt.subplot(224)
	plt.plot(gbp_sma5)
	plt.title('GBP')
	plt.savefig('SMA5.png')

	# plot last-day-closing-price
	plt.figure(5)
	plt.subplot(221)
	plt.plot(cny_lDay)
	plt.title('CNY')
	plt.subplot(222)
	plt.plot(jpy_lDay)
	plt.title('JPY')
	plt.subplot(223)
	plt.plot(eur_lDay)
	plt.title('EUR')
	plt.subplot(224)
	plt.plot(gbp_lDay)
	plt.title('GBP')
	plt.savefig('lastDay.png')
	
	X = construct_X(cny_data)
	print (X)
	
	



